<a href="https://colab.research.google.com/github/praffuln/generative-ai/blob/master/rag/rag-1/rag_indexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('hello world!!')

hello world!!


# Indexing

# Multi-representation Indexing

**Enviornment**

In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain-chroma langchain youtube-transcript-api pytube google-generativeai langchain_google_genai langchain_community


# Setup Keys For GOOGLE_API_KEY AND LANGSMITH

In [ ]:
# Assigning value to variable
GOOGLE_API_KEY=''
LANGCHAIN_API_KEY = ''

# Imports os, google Gemini EMbedding and Chat Classes with FAISS vectorstores

In [ ]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS



# setup langsmith for tracing

In [ ]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

## Initialize LLM And Embedding

In [ ]:
# LLM with function call
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)

# Create embeddings using a Google Generative AI model
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")



### Fetch the documents

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

loader = WebBaseLoader("https://lilianweng.github.io/posts/2024-02-05-human-data-quality/")
docs.extend(loader.load())

print(len(docs))
print('\nprinting metadata...\n')
for doc in docs:
  print(doc.metadata)



2

printing metadata...

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en'}
{'source': 'https://lilianweng.github.io/posts/2024-02-05-human-data-quality/', 'title': "Thinking about High-Quality Human Data | Lil'Log", 'description': '[Special thank you to Ian Kivlichan for many useful pointers (E.g. the 100+ year old Nature paper “Vox populi”) and nice feedback. \uf8ffüôè ]\nHigh-quality data is the fu

In [ ]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | llm
    | StrOutputParser()
)

summaries = chain.batch(docs, {"max_concurrency": 5})

len(summaries)
for summary in summaries:
  print(summary)
  print('\n\n\n\n\n\n\n')

#Store summary in vectorstore and original Docs in byteStore

In [ ]:
from langchain.storage import InMemoryByteStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_chroma import Chroma

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries",
                     embedding_function=embeddings)

# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# The retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

# Docs linked to summaries
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]


summary_docs

[Document(metadata={'doc_id': '9d570c6f-5a6f-4478-b18f-c8131a41d52b'}, page_content='This document explores the concept of LLM-powered autonomous agents, which are systems using large language models (LLMs) as their core controllers. It breaks down the key components of such agents: planning, memory, and tool use.\n\n**Planning** involves breaking down complex tasks into smaller subgoals and reflecting on past actions to improve future performance. Techniques like Chain of Thought (CoT), Tree of Thoughts, and ReAct are discussed for task decomposition and self-reflection.\n\n**Memory** encompasses short-term memory (in-context learning) and long-term memory (external vector stores). The document delves into the concept of Maximum Inner Product Search (MIPS) and various algorithms like LSH, ANNOY, HNSW, FAISS, and ScaNN for fast retrieval from vector stores.\n\n**Tool Use** equips LLMs with the ability to interact with external APIs and tools. Examples like MRKL, TALM, Toolformer, ChatG

In [ ]:
# Add
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

## Retrive documents

In [ ]:
### Get similarity search

query = "Memory in agents"
sub_docs = vectorstore.similarity_search(query,k=1)
sub_docs[0]


Document(metadata={'doc_id': 'c97f0a47-9843-44e2-af16-82ee4f19f65c'}, page_content='This document explores the concept of LLM-powered autonomous agents, which are systems using large language models (LLMs) as their core controllers. It breaks down the key components of such agents: planning, memory, and tool use.\n\n**Planning** involves breaking down complex tasks into smaller subgoals and reflecting on past actions to improve future performance. Techniques like Chain of Thought (CoT), Tree of Thoughts, and ReAct are discussed for task decomposition and self-reflection.\n\n**Memory** encompasses short-term memory (in-context learning) and long-term memory (external vector stores). The document delves into the concept of Maximum Inner Product Search (MIPS) and various algorithms like LSH, ANNOY, HNSW, FAISS, and ScaNN for fast retrieval from vector stores.\n\n**Tool Use** equips LLMs with the ability to interact with external APIs and tools. Examples like MRKL, TALM, Toolformer, ChatGP

In [ ]:
### Get complete document

retrieved_docs = retriever.get_relevant_documents(query,n_results=1)
retrieved_docs[0].page_content[0:500]


"\n\n\n\n\n\nLLM Powered Autonomous Agents | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nAgent System Overview\n\nComponent One: Planning\n\nTask Decomposition\n\nSelf-Reflection\n\n\nComponent Two: Memory\n\nTypes of Memory\n\nMaximum Inner Product Search (MIPS)\n\n"

# RAPTOR

https://colab.research.google.com/github/praffuln/generative-ai/blob/master/rag/rag-1/RAPTOR.ipynb#scrollTo=arHfZItPk-fr

# ColBERT

RAGatouille makes it as simple to use ColBERT.

ColBERT generates a contextually influenced vector for each token in the passages.

ColBERT similarly generates vectors for each token in the query.

Then, the score of each document is the sum of the maximum similarity of each query embedding to any of the document embeddings:

See here and here and here.


https://www.analyticsvidhya.com/blog/2024/04/colbert-improve-retrieval-performance-with-token-level-vector-embeddings/



In [ ]:
! pip install -U ragatouille


In [ ]:
from ragatouille import RAGPretrainedModel
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")


In [3]:
import requests

def get_wikipedia_page(title: str):
    """
    Retrieve the full text content of a Wikipedia page.

    :param title: str - Title of the Wikipedia page.
    :return: str - Full text content of the page as raw string.
    """
    # Wikipedia API endpoint
    URL = "https://en.wikipedia.org/w/api.php"

    # Parameters for the API request
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "explaintext": True,
    }

    # Custom User-Agent header to comply with Wikipedia's best practices
    headers = {"User-Agent": "RAGatouille_tutorial/0.0.1 (ben@clavie.eu)"}

    response = requests.get(URL, params=params, headers=headers)
    data = response.json()

    # Extracting page content
    page = next(iter(data["query"]["pages"].values()))
    return page["extract"] if "extract" in page else None

full_document = get_wikipedia_page("Hayao_Miyazaki")


In [4]:
RAG.index(
    collection=[full_document],
    index_name="Miyazaki-123",
    max_document_length=180,
    split_documents=True,
)


---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Aug 20, 09:13:52] #> Creating directory .ragatouille/colbert/indexes/Miyazaki-123 


[Aug 20, 09:13:53] [0] 		 #> Encoding 79 passages..


  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
100%|██████████| 3/3 [00:54<00:00, 18.10s/it]

[Aug 20, 09:14:48] [0] 		 avg_doclen_est = 132.35443115234375 	 len(local_sample) = 79
[Aug 20, 09:14:48] [0] 		 Creating 1,024 partitions.
[Aug 20, 09:14:48] [0] 		 *Estimated* 10,456 embeddings.
[Aug 20, 09:14:48] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/Miyazaki-123/plan.json ..


used 18 iterations (4.2399s) to cluster 9934 items into 1024 clusters
[0.036, 0.042, 0.039, 0.036, 0.032, 0.035, 0.033, 0.038, 0.035, 0.034, 0.034, 0.035, 0.035, 0.037, 0.038, 0.038, 0.033, 0.035, 0.035, 0.04, 0.038, 0.036, 0.032, 0.038, 0.038, 0.032, 0.035, 0.034, 0.037, 0.035, 0.038, 0.039, 0.037, 0.036, 0.033, 0.032, 0.036, 0.035, 0.034, 0.04, 0.034, 0.04, 0.034, 0.032, 0.037, 0.032, 0.034, 0.036, 0.036, 0.031, 0.032, 0.034, 0.032, 0.036, 0.034, 0.036, 0.041, 0.037, 0.036, 0.031, 0.034, 0.034, 0.035, 0.033, 0.036, 0.035, 0.037, 0.036, 0.032, 0.034, 0.036, 0.033, 0.034, 0.035, 0.037, 0.033, 0.033, 0.037, 0.037, 0.034, 0.034, 0.04, 0.033, 0.038, 0.034, 0.036, 0.036, 0.037, 0.034, 0.04, 0.035, 0.036, 0.035, 0.039, 0.035, 0.033, 0.038, 0.037, 0.037, 0.038, 0.039, 0.04, 0.036, 0.036, 0.036, 0.034, 0.037, 0.033, 0.035, 0.031, 0.034, 0.037, 0.038, 0.032, 0.037, 0.034, 0.036, 0.037, 0.035, 0.037, 0.035, 0.035, 0.033, 0.035, 0.034, 0.036, 0.036, 0.036]


0it [00:00, ?it/s]

...


100%|██████████| 3/3 [00:50<00:00, 16.87s/it]
1it [00:50, 50.96s/it]
100%|██████████| 1/1 [00:00<00:00, 688.38it/s]

[Aug 20, 09:15:43] #> Optimizing IVF to store map from centroids to list of pids..
[Aug 20, 09:15:43] #> Building the emb2pid mapping..
[Aug 20, 09:15:43] len(emb2pid) = 10456



100%|██████████| 1024/1024 [00:00<00:00, 21846.78it/s]

[Aug 20, 09:15:43] #> Saved optimized IVF to .ragatouille/colbert/indexes/Miyazaki-123/ivf.pid.pt
Done indexing!


'.ragatouille/colbert/indexes/Miyazaki-123'

In [5]:
results = RAG.search(query="What animation studio did Miyazaki found?", k=3)
results


Loading searcher for index Miyazaki-123 for the first time... This may take a few seconds
[Aug 20, 09:17:33] #> Loading codec...
[Aug 20, 09:17:33] #> Loading IVF...
[Aug 20, 09:17:33] Loading segmented_lookup_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Aug 20, 09:18:10] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 3429.52it/s]

[Aug 20, 09:18:10] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 271.76it/s]

[Aug 20, 09:18:10] Loading filter_pids_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


[Aug 20, 09:18:44] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . What animation studio did Miyazaki found?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  7284,  2996,  2106,  2771,  3148, 18637,  2179,
         1029,   102,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


[{'content': 'Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, Japanese: [mijaꜜzaki hajao]; born January 5, 1941) is a Japanese animator, filmmaker, and manga artist. A founder of Studio Ghibli, he has attained international acclaim as a masterful storyteller and creator of Japanese animated feature films, and is widely regarded as one of the most accomplished filmmakers in the history of animation.\nBorn in Tokyo City in the Empire of Japan, Miyazaki expressed interest in manga and animation from an early age, and he joined Toei Animation in 1963. During his early years at Toei Animation he worked as an in-between artist and later collaborated with director Isao Takahata.',
  'score': 25.629735946655273,
  'rank': 1,
  'document_id': 'e2ae4a3f-2225-4e0c-9e6c-e7fc71fc6dfa',
  'passage_id': 0},
 {'content': "In April 1984, Miyazaki opened his own office in Suginami Ward, naming it Nibariki.\n\n\n=== Studio Ghibli ===\n\n\n==== Early films (1985–1996) ====\nOn June 15, 1985, Miyazaki and Ta

In [6]:
retriever = RAG.as_langchain_retriever(k=3)
retriever.invoke("What animation studio did Miyazaki found?")


/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


[Document(page_content='Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, Japanese: [mijaꜜzaki hajao]; born January 5, 1941) is a Japanese animator, filmmaker, and manga artist. A founder of Studio Ghibli, he has attained international acclaim as a masterful storyteller and creator of Japanese animated feature films, and is widely regarded as one of the most accomplished filmmakers in the history of animation.\nBorn in Tokyo City in the Empire of Japan, Miyazaki expressed interest in manga and animation from an early age, and he joined Toei Animation in 1963. During his early years at Toei Animation he worked as an in-between artist and later collaborated with director Isao Takahata.'),
 Document(page_content="In April 1984, Miyazaki opened his own office in Suginami Ward, naming it Nibariki.\n\n\n=== Studio Ghibli ===\n\n\n==== Early films (1985–1996) ====\nOn June 15, 1985, Miyazaki and Takahata founded the animation production company Studio Ghibli as a subsidiary of Tokuma Shoten. Stud